In [132]:
# import libraries
import random
from collections import deque
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Keras libraries
from tensorflow.keras.models import Sequential

In [133]:
PRECEDING_PRICES = 60  # preceeding 60 days prices
FUTURE_PRICES_PREDICT = 3  # how far into the future to predict
TO_PREDICT = "Bitcoin"

In [134]:
# read datasets and join them
data_directory: Path = Path("./cryptocurrency-historical-prices")
main_df = pd.DataFrame()
for file in data_directory.iterdir():
    if file.is_file() and file.suffix == ".csv":
        cryptocurrency_name = file.name.split("_")[1].split(".")[
            0
        ]  # Bitcoin, Aave, Cardano, Solana, Dogecoin, etc.
    df = pd.read_csv(f"{data_directory.name}/{file.name}", names = ["SNo", "Name", "Symbol", "Date", "High", "Low", "Open", "Close", "Volume", "Marketcap"])
    # rename to distinguish the cryptocurrency we are working with
    df.rename(columns={"Close": f"{cryptocurrency_name}_close", "Volume": f"{cryptocurrency_name}_volume"}, inplace=True)
    # set time as index so we can join the dataframes
    df.set_index("Date", inplace=True)
    # drop columns we are not interested in
    df = df[[f"{cryptocurrency_name}_close", f"{cryptocurrency_name}_volume"]]
    main_df = df if len(main_df) == 0 else main_df.join(df)
    main_df = main_df.ffill() # propagate[s] last valid observation forward to next valid
    main_df = main_df.dropna()
    # main_df[f"{cryptocurrency_name}_close"] = main_df[f"{cryptocurrency_name}_close"].astype(float)
    # main_df[f"{cryptocurrency_name}_volume"] = main_df[f"{cryptocurrency_name}_volume"].astype(float)
    print(main_df.head())

                                  NEM_close       NEM_volume
Date                                                        
Date                                  Close           Volume
2015-04-02 23:59:59   0.0003137259918730706  28549.400390625
2015-04-03 23:59:59   0.0003102729970123619  20677.900390625
2015-04-04 23:59:59  0.00027652998687699437   23550.19921875
2015-04-05 23:59:59  0.00023193200468085706   26800.19921875
                                  NEM_close       NEM_volume EOS_close  \
Date                                                                     
Date                                  Close           Volume     Close   
2015-04-02 23:59:59   0.0003137259918730706  28549.400390625     Close   
2015-04-03 23:59:59   0.0003102729970123619  20677.900390625     Close   
2015-04-04 23:59:59  0.00027652998687699437   23550.19921875     Close   
2015-04-05 23:59:59  0.00023193200468085706   26800.19921875     Close   

                    EOS_volume  
Date                 

In [135]:
def make_order_decision(current_price, future_price):
	try:
		if float(current_price) < float(future_price): 
			return 1
		else:
			return 0
	except ValueError:
		print("Error: make_order_decision")

In [136]:
# create column with the target future value
main_df["future_price_to_predict"] = main_df[f"{TO_PREDICT}_close"].shift(-FUTURE_PRICES_PREDICT) # negative to shift columnn up
# main_df["future_price_to_predict"] = main_df["future_price_to_predict"].astype(float)
main_df["order_decision"] = list(map(make_order_decision, main_df[f"{TO_PREDICT}_close"], main_df["future_price_to_predict"]))
main_df = main_df.dropna()
print(main_df.head())

Error: make_order_decision
                                  NEM_close       NEM_volume EOS_close  \
Date                                                                     
2015-04-02 23:59:59   0.0003137259918730706  28549.400390625     Close   
2015-04-03 23:59:59   0.0003102729970123619  20677.900390625     Close   
2015-04-04 23:59:59  0.00027652998687699437   23550.19921875     Close   
2015-04-05 23:59:59  0.00023193200468085706   26800.19921875     Close   
2015-04-06 23:59:59  0.00028870601090602577          22511.5     Close   

                    EOS_volume        Monero_close  Monero_volume  \
Date                                                                
2015-04-02 23:59:59     Volume  0.9692599773406982   88831.296875   
2015-04-03 23:59:59     Volume  0.9522449970245361       139293.0   
2015-04-04 23:59:59     Volume  0.8592900037765503       128977.0   
2015-04-05 23:59:59     Volume  0.8837310075759888       143439.0   
2015-04-06 23:59:59     Volume  0.789462

In [ ]:

# train test split
sorted_dates = sorted(main_df.index.values)
last_5_percent = sorted_dates[-int(len(sorted_dates) * 0.05)]

t = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

x_train, y_train = preprocess_df(main_df)
x_test, y_test = preprocess_df(validation_main_df)

print(f"train data: {len(x_train)} validation: {len(x_test)}")
print(f"Dont buys: {y_train.count(0)}, buys: {y_train.count(1)}")
print(f"VALIDATION Dont buys: {y_test.count(0)}, buys: {y_test.count(1)}")